#Bibliotecas e Drives

In [16]:
import pandas as pd
import numpy as np
import io
import requests
import folium
from folium import plugins
from datetime import date
%matplotlib inline

In [2]:
%cd /content/drive/My\ Drive/Colab\ Notebooks/covid19

/content/drive/My Drive/Colab Notebooks/covid19


In [3]:
!ls

 API_SP.POP.TOTL_DS2_en_csv_v2_887275.csv
 brasil_20200324.csv
 brasil_20200326.csv
 Brasil_20200327.html
 Brasil_covid19-calor.html
 Brasil_covid19.html
 covid19BrazilCelular-script.ipynb
 covid19Brazil-script.ipynb
 covid19-script.ipynb
 covid19-scripts-trash.ipynb
 covid19-SEIR.ipynb
'Documento sem título.gdoc'
 full_data_20200320.csv
 full_data_20200321.csv
 full_data_20200322.csv
 full_data_20200323.csv
 full_data_20200326.csv
 Metadata_Country_API_SP.POP.TOTL_DS2_en_csv_v2_887275.csv
 Metadata_Indicator_API_SP.POP.TOTL_DS2_en_csv_v2_887275.csv
 teste.png


#Carregando o banco de dados

In [4]:
url='https://raw.githubusercontent.com/wcota/covid19br/master/cases-gps.csv'
data = requests.get(url).content
brazil_df = pd.read_csv(io.StringIO(data.decode('utf-8')))
brazil_df

,type,name,lat,lon,total,total_per_100k_inhabitants
0,1,Abadia de Goiás/GO,-16.757264,-49.441221,183,2085.94551
1,D1,Abadia de Goiás/GO,-16.757264,-49.441221,1,11.39861
2,1,Abadia dos Dourados/MG,-18.491119,-47.406359,44,629.56074
3,D1,Abadia dos Dourados/MG,-18.491119,-47.406359,1,14.30820
4,1,Abadiânia/GO,-16.194095,-48.706777,77,384.19319
...,...,...,...,...,...,...
9149,1,Áurea/RS,-27.697092,-52.052178,12,337.64772
9150,1,Ângulo/PR,-23.195835,-51.909498,17,580.60109
9151,1,Óbidos/PA,-1.901072,-55.520812,933,1789.51608
9152,D1,Óbidos/PA,-1.901072,-55.520812,23,44.11454


In [5]:
brazil_df['mortes'] = pd.np.where(brazil_df['type'].str.contains('D1'), brazil_df['total'], 0)
brazil_df['total'] = pd.np.where(brazil_df['type'].str.contains('D1'), 0, brazil_df['total'])

brazil_df.head(15)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
  


,type,name,lat,lon,total,total_per_100k_inhabitants,mortes
0,1,Abadia de Goiás/GO,-16.757264,-49.441221,183,2085.94551,0
1,D1,Abadia de Goiás/GO,-16.757264,-49.441221,0,11.39861,1
2,1,Abadia dos Dourados/MG,-18.491119,-47.406359,44,629.56074,0
3,D1,Abadia dos Dourados/MG,-18.491119,-47.406359,0,14.30820,1
4,1,Abadiânia/GO,-16.194095,-48.706777,77,384.19319,0
5,D1,Abadiânia/GO,-16.194095,-48.706777,0,39.91618,8
6,1,Abaetetuba/PA,-1.721828,-48.878843,2772,1757.79021,0
7,D1,Abaetetuba/PA,-1.721828,-48.878843,0,69.11946,109
8,1,Abaeté/MG,-19.156683,-45.448121,26,111.89052,0
9,1,Abaiara/CE,-7.360781,-39.048788,60,511.20389,0


In [20]:
today = date.today()

hj = today.strftime("%d/%m/%Y")

brazil_df['total'].unique()

array([ 183,    0,   44, ..., 2901,  573, 1449])

## Plotando sobre o mapa


In [23]:
#Definindo o tamanho dos circulos
brazil_df['size']=brazil_df['total'].apply(lambda total:12 if total>=900 else
                                           10 if total>=400 and total<900 else 
                                           9 if total>=200 and total<400 else 3)

#plotando o mapa, em PB
mapa = folium.Map(location=[-15.788497,-47.879873],tiles='CartoDB dark_matter', 
                  zoom_start=4)

total_casos = 0
for i in brazil_df['total']:
  total_casos = total_casos + i

total_obitos = 0
for i in brazil_df['mortes']:
  total_obitos = total_obitos + i

#Marcação dos pontos
for lat,lon,cidade,total,mortes,size,tipo in zip(brazil_df['lat'], brazil_df['lon'],
                                                 brazil_df['name'], brazil_df['total'],
                                                 brazil_df['mortes'],brazil_df['size'],
                                                 brazil_df['type']):
  if 'NÃO' in cidade:
    texto = "{}<br> Casos: {:,}"
    cor = 'orange'
    texto = texto.format(cidade, total)
  elif (tipo == 'D1') or (tipo == 'D0') :
    texto = "{}<br> Óbitos: {:,}"
    cor = 'black'
    texto = texto.format(cidade, mortes)
    size = 0.1
  else:
    texto = "{}<br> Casos: {:,}"
    cor = 'red'
    texto = texto.format(cidade, total)

  folium.CircleMarker([lat,lon], radius=size, popup=texto, color=cor, 
                      fill=True, fill_opacity=0.9, fill_color='darkred').add_to(mapa)

txt = "Total no Brasil<br>de casos: {:,} <br>de mortes: {:,} <br>data: {:}"
txt = txt.format(total_casos, total_obitos, hj)
popup = folium.Popup(txt, max_width=300)
icon = folium.Icon(color='black', icon='info-sign')
marcador = folium.Marker(location=[-15.6914,-25.8008], popup=popup, icon=icon)

mapa.add_child(marcador)

mapa.save('Brasil_covid19.html')
#mapa

In [25]:
#plotando o mapa, em PB
mapa = folium.Map(location=[-15.788497,-47.879873],tiles='CartoDB dark_matter', 
                  zoom_start=4)

txt = "Total no Brasil<br>de casos: {:,} <br>de mortes: {:,} <br>data: {:}"
txt = txt.format(total_casos, total_obitos, hj)
popup = folium.Popup(txt, max_width=300)
icon = folium.Icon(color='black', icon='info-sign')
marcador = folium.Marker(location=[-15.6914,-25.8008], popup=popup, icon=icon)

mapa.add_child(marcador)

#Mapa de calor
mapa_calor = brazil_df[['lat', 'lon', 'total']].values #.as_matrix()
mapa.add_child(plugins.HeatMap(mapa_calor, radius=15))


mapa.save('Brasil_covid19-calor.html')
#mapa